In [1]:
!git clone https://github.com/Olyco/EasyTSF.git

Cloning into 'EasyTSF'...
remote: Enumerating objects: 257, done.
remote: Counting objects: 100% (257/257), done.
remote: Compressing objects: 100% (168/168), done.
remote: Total 257 (delta 125), reused 199 (delta 75), pack-reused 0 (from 0)
Receiving objects: 100% (257/257), 1.51 MiB | 11.29 MiB/s, done.
Resolving deltas: 100% (125/125), done.


In [1]:
!pip install -r "EasyTSF/requirements.txt"

In [19]:
!python EasyTSF/train.py -c config/reproduce_conf/RMoK/ETTh1_96for96.py

Traceback (most recent call last):
  File "/content/EasyTSF/train.py", line 9, in <module>
    from ray.tune.integration.pytorch_lightning import TuneReportCheckpointCallback
ModuleNotFoundError: No module named 'ray'


In [17]:
!ls

EasyTSF  sample_data


In [16]:
!cd EasyTSF/

In [18]:
!pwd

/content
